In [86]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [4]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [6]:
soup = BeautifulSoup(source, 'xml')

In [7]:
table=soup.find('table')

In [8]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [9]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [10]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
df=df[df['Borough']!='Not assigned']

In [12]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [13]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [14]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [15]:
df_merge.drop_duplicates(inplace=True)

In [16]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [17]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [18]:
df_merge.shape

(103, 3)

In [29]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [30]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [31]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [40]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

In [41]:
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [46]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [47]:
CLIENT_ID = 'ENNZUXAOREVESSHOJ0LWAT2BUG5OPEPX04CW2LVHUZTAPFKE' # your Foursquare ID
CLIENT_SECRET = '1V00JTQVVPU1TZ3ZNVN5A0NE2CA43XSITUDRPA4ND2RB5YDT' # your Foursquare Secret
VERSION = '20180604'

In [52]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=50
    LIMIT=50
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [53]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [54]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Davisville,43.704324,-79.388790,Jules Cafe Patisserie,43.704138,-79.388413,Dessert Shop
1,Davisville,43.704324,-79.388790,Thobors Boulangerie Patisserie Café,43.704514,-79.388616,Café
2,Davisville,43.704324,-79.388790,Positano,43.704558,-79.388639,Italian Restaurant
3,Davisville,43.704324,-79.388790,Second Cup,43.704344,-79.388659,Coffee Shop
4,Rosedale,43.679563,-79.377529,Park Drive Reservation Lands,43.679822,-79.377787,Park


In [55]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",1,1,1,1,1,1
Christie,1,1,1,1,1,1
"Commerce Court, Victoria Hotel",5,5,5,5,5,5
Davisville,4,4,4,4,4,4
"First Canadian Place, Underground city",6,6,6,6,6,6
"Kensington Market, Chinatown, Grange Park",4,4,4,4,4,4
"Little Portugal, Trinity",2,2,2,2,2,2
"Regent Park, Harbourfront",1,1,1,1,1,1
"Richmond, Adelaide, King",3,3,3,3,3,3


In [70]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(40, 26)

In [71]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Art Gallery,Bakery,Bookstore,Breakfast Spot,Building,Café,Cocktail Bar,Coffee Shop,Cupcake Shop,...,Italian Restaurant,Japanese Restaurant,Liquor Store,Nightclub,Park,Performing Arts Venue,Restaurant,Salad Place,Thrift / Vintage Store,Yoga Studio
0,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,...,0.00,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0
1,Christie,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,...,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,"Commerce Court, Victoria Hotel",0.2,0.000000,0.2,0.0,0.000000,0.000000,0.0,0.20,0.000000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Davisville,0.0,0.000000,0.0,0.0,0.000000,0.250000,0.0,0.25,0.000000,...,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,"First Canadian Place, Underground city",0.0,0.166667,0.0,0.0,0.166667,0.166667,0.0,0.00,0.166667,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0


In [72]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [76]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CN Tower, King and Spadina, Railway Lands, Har...",Performing Arts Venue,Yoga Studio,Gastropub,Bakery,Bookstore,Breakfast Spot,Building,Café,Cocktail Bar,Coffee Shop
1,Christie,Nightclub,Yoga Studio,Gastropub,Bakery,Bookstore,Breakfast Spot,Building,Café,Cocktail Bar,Coffee Shop
2,"Commerce Court, Victoria Hotel",Art Gallery,Bookstore,Coffee Shop,Deli / Bodega,Gym,Gastropub,Bakery,Breakfast Spot,Building,Café
3,Davisville,Café,Coffee Shop,Italian Restaurant,Dessert Shop,Yoga Studio,Gastropub,Bakery,Bookstore,Breakfast Spot,Building
4,"First Canadian Place, Underground city",Gastropub,Salad Place,Bakery,Building,Café,Cupcake Shop,Yoga Studio,Bookstore,Breakfast Spot,Cocktail Bar


In [77]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 4, 0, 0, 0, 0, 0, 3, 0, 0])

In [78]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,M4M,East Toronto,Studio District,43.659526,-79.340923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,"CN Tower, King and Spadina, Railway Lands, Har...",Performing Arts Venue,Yoga Studio,Gastropub,Bakery,Bookstore,Breakfast Spot,Building,Café,Cocktail Bar,Coffee Shop
1,4,Christie,Nightclub,Yoga Studio,Gastropub,Bakery,Bookstore,Breakfast Spot,Building,Café,Cocktail Bar,Coffee Shop
2,0,"Commerce Court, Victoria Hotel",Art Gallery,Bookstore,Coffee Shop,Deli / Bodega,Gym,Gastropub,Bakery,Breakfast Spot,Building,Café
3,0,Davisville,Café,Coffee Shop,Italian Restaurant,Dessert Shop,Yoga Studio,Gastropub,Bakery,Bookstore,Breakfast Spot,Building
4,0,"First Canadian Place, Underground city",Gastropub,Salad Place,Bakery,Building,Café,Cupcake Shop,Yoga Studio,Bookstore,Breakfast Spot,Cocktail Bar
